# Superswaps

In [ ]:
#| default_exp superswap

In [ ]:
#| export

import json, requests
from sugar.swap import build_super_swap_data, SuperSwapData, SuperSwapQuote, setup_planner, SuperSwapDataInput
from sugar.token import Token
from sugar.quote import Quote
from sugar.helpers import get_salt, serialize_ica_calls
from sugar.config import hyperlane_relay_url, hyperlane_relayers
from sugar.chains import get_chain_from_token, get_async_chain_from_token, AsyncChain, Chain, AsyncOPChain, OPChain
from typing import List, Dict, Any, Optional, Union
from abc import ABC, abstractmethod

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| export

domains_abi = [
    {
        "name": "domains",
        "type": "function",
        "stateMutability": "view",
        "inputs": [
            {
                "name": "",
                "type": "uint256"
            }
        ],
        "outputs": [
            {
                "name": "domain",
                "type": "uint256"
            }
        ]
    }
]

# TODO: remove this when domains are supported on all chains
async def get_domain_async(chain_id: int) -> int:
    # TODO: remove chain_id arg when all chains support domains
    async with AsyncOPChain() as op:
        contract = op.web3.eth.contract(address=op.settings.message_module_contract_addr, abi=domains_abi)
        domain = await contract.functions.domains(chain_id).call()
        # TODO: remove fallback to chain_id when all chains support domains
        return domain if domain != 0 else int(chain_id)

def get_domain(chain_id: int) -> int:
    # TODO: remove chain_id arg when all chains support domains
    with OPChain() as op:
        contract = op.web3.eth.contract(address=op.settings.message_module_contract_addr, abi=domains_abi)
        domain = contract.functions.domains(chain_id).call()
        # TODO: remove fallback to chain_id when all chains support domains
        return domain if domain != 0 else int(chain_id)

In [ ]:
#| export

class SuperswapRelayer(ABC):
    @abstractmethod
    def share_calls(self, calls: List[dict], salt: str, commitment_dispatch_tx: str, origin_domain: int) -> None:
        """Share calls with the relayer."""
        pass

# TODO: add helper to inspect tx using https://explorer.hyperlane.xyz/?search

class HTTPSuperswapRelayer(SuperswapRelayer):
    """HTTP-based relayer implementation."""
    def share_calls(self, calls: List[dict], salt: str, commitment_dispatch_tx: str, origin_domain: int) -> None:
        """
        Share calls with private relayer.
        
        Args:
            calls: List of call data dictionaries
            salt: Hex string salt value
            commitment_dispatch_tx: Transaction hash string
            origin_domain: Domain number
        """
        body = json.dumps({
            'commitmentDispatchTx': commitment_dispatch_tx,
            'originDomain': origin_domain,
            'calls': calls,
            'salt': salt,
            'relayers': hyperlane_relayers
        })
        resp = requests.post(hyperlane_relay_url, headers={'Content-Type': 'application/json'}, data=body)
        print(f"Hyperlane response: {resp.status_code}: {resp.text}")
        if not resp.ok:
            response_text = resp.text
            error_msg = f"Failed to share calls with relayer: {resp.status_code} {response_text}"
            print(f"Error: {error_msg}")
            raise Exception(error_msg)
                

## Common superswap base 

In [ ]:
#| export

supported_chains = ["OP", "Lisk", "Uni"]

class SuperSwapCommon:
    amount_in: Optional[float] = None
    from_token: Optional[Token] = None
    to_token: Optional[Token] = None
    from_chain: Optional[Union[Chain, AsyncChain]] = None
    to_chain: Optional[Union[Chain, AsyncChain]] = None
    origin_bridge_token: Optional[Token] = None
    destination_bridge_token: Optional[Token] = None
    origin_quote: Optional[Quote] = None
    destination_quote: Optional[Quote] = None

    @property
    def starts_with_bridge_token(self): return self.from_token.token_address == self.origin_bridge_token.token_address

    @property
    def bridged_amount(self) -> float: 
        assert self.amount_in is not None, "amount_in must be set before accessing bridged_amount"
        if not self.starts_with_bridge_token:
            return float(self.origin_quote.amount_out / 10 ** self.origin_bridge_token.decimals)
        else:
            print(f">>>>>>>>>> serving bridged amount as {self.amount_in} because we start with bridge token")
            return float(self.amount_in / 10 ** self.origin_bridge_token.decimals)

    @property
    def ends_with_bridge_token(self): return self.to_token.token_address == self.destination_bridge_token.token_address

    def check_chain_support(self, from_token: Token, to_token: Token) -> None:
        """Check if the given tokens are supported for superswap."""
        from_chain, to_chain = get_async_chain_from_token(from_token), get_async_chain_from_token(to_token)
        if from_chain.name not in supported_chains or to_chain.name not in supported_chains:
            raise ValueError(f"Superswap only supports {supported_chains}. Got {from_chain.name} -> {to_chain.name}")

    def init(self, from_token: Token, to_token: Token, amount_in: float, sync: bool = True):
        """Shared prep for sync and async versions."""

        self.from_token, self.to_token, self.amount_in = from_token, to_token, amount_in

        print(f"inittialied {self.from_token.symbol} -> {self.to_token.symbol} with amount {self.amount_in}")

        if sync:
            self.from_chain, self.to_chain = get_chain_from_token(from_token), get_chain_from_token(to_token)
        else:    
            self.from_chain, self.to_chain = get_async_chain_from_token(from_token), get_async_chain_from_token(to_token)

    def prepare_super_quote(self) -> SuperSwapQuote:
        return SuperSwapQuote(
            from_token=self.from_token,
            to_token=self.to_token,
            from_bridge_token=self.origin_bridge_token,
            to_bridge_token=self.destination_bridge_token,
            amount_in=self.amount_in,
            origin_quote=self.origin_quote,
            destination_quote=self.destination_quote
        )

    def prepare_super_swap(self, user_ica_address: str, user_ICA_balance: int, origin_domain: int, destination_domain: int,
        origin_hook: str, slippage: float, bridge_fee: int, xchain_fee: int, salt: Optional[str] = None) -> None:
        swap_data = build_super_swap_data(SuperSwapDataInput(
            from_token=self.from_token,
            to_token=self.to_token,
            from_bridge_token=self.origin_bridge_token,
            to_bridge_token=self.destination_bridge_token,
            account=self.from_chain.account.address,
            user_ICA=user_ica_address,
            user_ICA_balance=user_ICA_balance,
            origin_domain=origin_domain,
            origin_bridge=self.from_chain.settings.bridge_contract_addr,
            origin_hook=origin_hook,
            origin_ICA_router=self.from_chain.settings.interchain_router_contract_addr,
            destination_ICA_router=self.to_chain.settings.interchain_router_contract_addr,
            destination_router=self.to_chain.settings.swapper_contract_addr,
            destination_domain=destination_domain,
            slippage=slippage,
            bridged_amount=self.bridged_amount,
            swapper_contract_addr=self.to_chain.settings.swapper_contract_addr,
            destination_quote=self.destination_quote,
            bridge_fee=bridge_fee,
            xchain_fee=xchain_fee,
            salt=salt if salt else get_salt()
        ))

        origin_planner = setup_planner(
            quote=self.origin_quote,
            slippage=slippage,
            # money goes to the universal router (aka swapper) for bridging
            account=self.from_chain.settings.swapper_contract_addr, 
            router_address=self.from_chain.settings.swapper_contract_addr
        ) if self.origin_quote else None

        cmds, inputs = "", [] 

        if origin_planner:
            cmds += origin_planner.commands
            inputs.extend(origin_planner.inputs)
        if swap_data.destination_planner:
            cmds += swap_data.destination_planner.commands.replace("0x", "") if cmds != "" else swap_data.destination_planner.commands
            inputs.extend(swap_data.destination_planner.inputs)

        return cmds, inputs, swap_data

    def prepare_write(self, total_fee: int) -> tuple[int, int]:
        assert self.amount_in is not None, "amount_in must be set before preparing write"
        value = self.amount_in * (10 ** self.from_token.decimals)
        # TODO: extend this to proper native token support
        message_fee = value + total_fee if self.from_token.wrapped_token_address else total_fee
        return value, message_fee

    def prepare_result(self, swap_data: SuperSwapData, tx) -> str:
        if swap_data.needs_relay:
            self.relayer.share_calls(
                calls=serialize_ica_calls(swap_data.calls),
                salt=swap_data.salt,
                commitment_dispatch_tx=f'0x{tx["transactionHash"].hex()}',
                origin_domain=swap_data.origin_domain
            )
        return f'0x{tx["transactionHash"].hex()}'


## Sync version of superswap

In [ ]:
#| export

class Superswap(SuperSwapCommon):
    def __init__(self, relayer: Optional[SuperswapRelayer] = None, chain_for_writes: Optional[Chain] = None):
        self.chain_for_writes = chain_for_writes
        self.relayer = relayer or HTTPSuperswapRelayer()

    def swap(self, from_token: Token, to_token: Token, amount: float, slippage: Optional[float] = None) -> str:
        self.check_chain_support(from_token, to_token)
        quote = self.get_super_quote(from_token=from_token, to_token=to_token, amount_in=amount)
        return self.swap_from_quote(quote=quote, slippage=slippage)

    def get_super_quote(self, from_token: Token, to_token: Token, amount_in: float) -> SuperSwapQuote:
        self.init(from_token, to_token, amount_in, sync=True)

        with self.from_chain, self.to_chain:
            self.origin_bridge_token = self.from_chain.get_superswap_connector_token()
            self.destination_bridge_token = self.to_chain.get_superswap_connector_token()

            # we only need origin quote if we don't start with oUSDT
            if not self.starts_with_bridge_token:
                self.origin_quote = self.from_chain.get_quote(from_token, self.origin_bridge_token, amount_in)
                assert self.origin_quote is not None, "No origin quote found"

            # we need destination quote if we don't end with oUSDT
            if not self.ends_with_bridge_token:
                self.destination_quote = self.to_chain.get_quote(self.destination_bridge_token, to_token, self.bridged_amount)
                assert self.destination_quote is not None, "No destination quote found"

            return self.prepare_super_quote()

    def swap_from_quote(self, quote: SuperSwapQuote, slippage: Optional[float] = None, salt: Optional[str] = None):
        self.check_chain_support(from_token, to_token)

        origin_quote, destination_quote = quote.origin_quote, quote.destination_quote

        with self.from_chain, self.to_chain:
            if not self.from_chain.account: raise ValueError("Cannot superswap without an account. Please connect your wallet first.")
            
            slippage = slippage if slippage is not None else self.from_chain.settings.swap_slippage
            
            # TODO: use chain.get_domain() when all chains support domains
            origin_domain = get_domain(int(self.from_chain.chain_id))
            destination_domain = get_domain(int(self.to_chain.chain_id))
            user_ica_address= self.from_chain.get_remote_interchain_account(destination_domain)
            bridge_fee = self.from_chain.get_bridge_fee(int(self.to_chain.chain_id))
            xchain_fee = self.from_chain.get_xchain_fee(destination_domain)            
            total_fee = bridge_fee + xchain_fee if quote.to_token.token_address != quote.to_bridge_token.token_address else bridge_fee 

            cmds, inputs, swap_data = self.prepare_super_swap(
                user_ica_address=user_ica_address,
                user_ICA_balance=self.to_chain.get_user_ica_balance(user_ica_address),
                origin_domain=origin_domain,
                destination_domain=destination_domain,
                origin_hook=self.from_chain.get_ica_hook(),
                slippage=slippage,
                bridge_fee=bridge_fee,
                xchain_fee=xchain_fee,
                salt=salt
            )

            return self.write(chain=self.chain_for_writes or self.from_chain, cmds=cmds, inputs=inputs, swap_data=swap_data, total_fee=total_fee)

    def write(self, chain: AsyncChain, swap_data: SuperSwapData, cmds: str, inputs: List[bytes], total_fee: int) -> str:
        value, message_fee = self.prepare_write(total_fee)
        with chain:
            chain.set_token_allowance(self.from_token, chain.settings.swapper_contract_addr, value)
            tx = chain.sign_and_send_tx(chain.swapper.functions.execute(*[cmds, inputs]), value=message_fee)
            return self.prepare_result(swap_data, tx)

## Async version of superswap

In [ ]:
#| export

class AsyncSuperswap(SuperSwapCommon):
    def __init__(self, relayer: Optional[SuperswapRelayer] = None, chain_for_writes: Optional[AsyncChain] = None):
        self.chain_for_writes = chain_for_writes
        self.relayer = relayer or HTTPSuperswapRelayer()    

    async def swap(self, from_token: Token, to_token: Token, amount: float, slippage: Optional[float] = None) -> str:
        self.check_chain_support(from_token, to_token)
        quote = await self.get_super_quote(from_token=from_token, to_token=to_token, amount_in=amount)
        return await self.swap_from_quote(quote=quote, slippage=slippage)

    async def get_super_quote(self, from_token: Token, to_token: Token, amount_in: float) -> SuperSwapQuote:
        self.init(from_token, to_token, amount_in, sync=False)

        async with self.from_chain, self.to_chain:
            self.origin_bridge_token = await self.from_chain.get_superswap_connector_token()
            self.destination_bridge_token = await self.to_chain.get_superswap_connector_token()

            # we only need origin quote if we don't start with oUSDT
            if not self.starts_with_bridge_token:
                self.origin_quote = await self.from_chain.get_quote(from_token, self.origin_bridge_token, amount_in)
                assert self.origin_quote is not None, "No origin quote found"

            # we need destination quote if we don't end with oUSDT
            if not self.ends_with_bridge_token:
                self.destination_quote = await self.to_chain.get_quote(self.destination_bridge_token, to_token, self.bridged_amount)
                assert self.destination_quote is not None, "No destination quote found"

            return self.prepare_super_quote()

    async def swap_from_quote(self, quote: SuperSwapQuote, slippage: Optional[float] = None, salt: Optional[str] = None):
        self.check_chain_support(from_token, to_token)

        async with self.from_chain, self.to_chain:
            if not self.from_chain.account: raise ValueError("Cannot superswap without an account. Please connect your wallet first.")
            
            slippage = slippage if slippage is not None else self.from_chain.settings.swap_slippage
            
            # TODO: use chain.get_domain() when all chains support domains
            origin_domain = await get_domain_async(int(self.from_chain.chain_id))
            destination_domain = await get_domain_async(int(self.to_chain.chain_id))
            user_ica_address=await self.from_chain.get_remote_interchain_account(destination_domain)
            bridge_fee = await self.from_chain.get_bridge_fee(int(self.to_chain.chain_id))
            xchain_fee = await self.from_chain.get_xchain_fee(destination_domain)            
            total_fee = bridge_fee + xchain_fee if quote.to_token.token_address != quote.to_bridge_token.token_address else bridge_fee 

            print(f"fees: bridge={bridge_fee}, xchain={xchain_fee}, total={total_fee}")

            cmds, inputs, swap_data = self.prepare_super_swap(
                user_ica_address=user_ica_address,
                user_ICA_balance=await self.to_chain.get_user_ica_balance(user_ica_address),
                origin_domain=origin_domain,
                destination_domain=destination_domain,
                origin_hook=await self.from_chain.get_ica_hook(),
                slippage=slippage,
                bridge_fee=bridge_fee,
                xchain_fee=xchain_fee,
                salt=salt
            )

            print(">>>>>>>> cmds:", cmds)
            print(">>>>>>>> inputs:", inputs)

            return await self.write(chain=self.chain_for_writes or self.from_chain, cmds=cmds, inputs=inputs, swap_data=swap_data, total_fee=total_fee)

    async def write(self, chain: AsyncChain, swap_data: SuperSwapData, cmds: str, inputs: List[bytes], total_fee: int) -> str:
        value, message_fee = self.prepare_write(total_fee)

        async with chain:
            await chain.set_token_allowance(self.from_token, chain.settings.swapper_contract_addr, value)

            

            print(f"set allowance >>>>> {self.from_token} {chain.settings.swapper_contract_addr} {value}")
            print("allownance is", await chain.check_token_allowance(self.from_token, chain.settings.swapper_contract_addr))


            # tx_function = chain.swapper.functions.execute(*[cmds, inputs])
            # built_tx = await tx_function.build_transaction({
            #     'value': message_fee,
            #     'from': chain.account.address,
            #     'gas': 2000000,  # adjust as needed
            #     'gasPrice': await chain.web3.eth.gas_price
            # })

            # # Inspect the payload
            # print(f"Transaction data (payload): {built_tx['data']}")
            # print(f"To address: {built_tx['to']}")
            # print(f"Value: {built_tx['value']}")
            # print(f"Full transaction: {built_tx}")

            tx = await chain.sign_and_send_tx(chain.swapper.functions.execute(*[cmds, inputs]), value=message_fee)
            return self.prepare_result(swap_data, tx)

## Tests

In [ ]:
from sugar.helpers import require_supersim
from fastcore.test import test_eq

require_supersim()

In [ ]:
class MockSuperswapRelayer(SuperswapRelayer):
    """Mock relayer implementation for testing."""
    
    def __init__(self):
        self.calls_history: List[Dict[str, Any]] = []
    
    def share_calls(self, calls: List[dict], salt: str, commitment_dispatch_tx: str, origin_domain: int) -> None:
        """Mock implementation that records calls for verification."""
        call_data = {
            'calls': calls,
            'salt': salt,
            'commitment_dispatch_tx': commitment_dispatch_tx,
            'origin_domain': origin_domain
        }
        self.calls_history.append(call_data)
        print(f"Mock relayer received call: {call_data}")
    
    def get_last_call(self) -> Optional[Dict[str, Any]]:
        """Get the most recent call data."""
        return self.calls_history[-1] if self.calls_history else None
    
    def get_call_count(self) -> int:
        """Get the total number of calls made."""
        return len(self.calls_history)

In [ ]:
from sugar import AsyncOPChainSimnet, OPChainSimnet, AsyncOPChain, AsyncBaseChain, AsyncLiskChain

# try unsupported chains

async with AsyncOPChainSimnet() as op_sim:
    from_token, to_token = AsyncOPChain.velo, AsyncBaseChain.aero 
    error = None
    try:
        tx = await AsyncSuperswap(chain_for_writes=op_sim, relayer=MockSuperswapRelayer()).swap(from_token, to_token, amount=20)
    except ValueError as e:
        error = e
    test_eq(str(error), "Superswap only supports ['OP', 'Lisk', 'Uni']. Got OP -> Base")

# # sync version
with OPChainSimnet() as op_sim:
    relayer=MockSuperswapRelayer()
    from_token, to_token = AsyncOPChain.velo, AsyncLiskChain.lsk
    tx = Superswap(chain_for_writes=op_sim, relayer=relayer).swap(from_token, to_token, amount=20)
    assert(tx.startswith("0x"))
    test_eq(relayer.get_call_count(), 1)
    last_call = relayer.get_last_call()
    test_eq(type(last_call["salt"]), str)
    test_eq(type(last_call["origin_domain"]), int)

# # async version
async with AsyncOPChainSimnet() as op_sim:
    relayer=MockSuperswapRelayer()
    from_token, to_token = AsyncOPChain.velo, AsyncLiskChain.lsk
    tx = await AsyncSuperswap(chain_for_writes=op_sim, relayer=relayer).swap(from_token, to_token, amount=20)
    assert(tx.startswith("0x"))
    test_eq(relayer.get_call_count(), 1)
    last_call = relayer.get_last_call()
    test_eq(type(last_call["salt"]), str)
    test_eq(type(last_call["origin_domain"]), int)

# # to o_usdt variation
async with AsyncOPChainSimnet() as op_sim:
    relayer=MockSuperswapRelayer()
    from_token, to_token = AsyncOPChain.velo, AsyncLiskChain.o_usdt
    tx = await AsyncSuperswap(chain_for_writes=op_sim, relayer=relayer).swap(from_token, to_token, amount=20)
    assert(tx.startswith("0x"))
    test_eq(relayer.get_call_count(), 0)

# from o_usdt variation
with OPChainSimnet() as op_sim:
    print(op_sim.settings)
    relayer=MockSuperswapRelayer()
    from_token, to_token = AsyncOPChain.o_usdt, AsyncLiskChain.lsk
    tx = Superswap(chain_for_writes=op_sim, relayer=relayer).swap(from_token, to_token, amount=1)
    assert(tx.startswith("0x"))
    test_eq(relayer.get_call_count(), 1)
    last_call = relayer.get_last_call()
    test_eq(type(last_call["salt"]), str)
    test_eq(type(last_call["origin_domain"]), int)



inittialied VELO -> LSK with amount 20
Mock relayer received call: {'calls': [{'to': '0x0000000000000000000000001217bfe6c773eec6cc4a38b5dc45b92292b6e189', 'value': '0', 'data': '0x095ea7b300000000000000000000000001d40099fcd87c018969b0e8d4ab1633fb34763cffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffff'}, {'to': '0x00000000000000000000000001d40099fcd87c018969b0e8d4ab1633fb34763c', 'value': '0', 'data': '0x24856bc3000000000000000000000000000000000000000000000000000000000000004000000000000000000000000000000000000000000000000000000000000000800000000000000000000000000000000000000000000000000000000000000002a1a100000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000020000000000000000000000000000000000000000000000000000000000000040000000000000000000000000000000000000000000000000000000000000032000000000000000000000000000000000000000000000000000000000000002c00000000000000000000000000000000000000000000000000000000000

In [ ]:
# real calls
# async with AsyncOPChain() as op:
#     from_token, to_token = AsyncOPChain.velo, AsyncLiskChain.eth
#     tx = await AsyncSuperswap().swap(from_token, to_token, amount=20)
#     print(f"Async swap tx: {tx}")

# async with AsyncLiskChain() as lisk:
#     from_token, to_token = AsyncLiskChain.lsk, AsyncOPChain.o_usdt
#     tx = await AsyncSuperswap().swap(from_token, to_token, amount=4)
#     print(f"Async swap tx: {tx}")


    
# from_token, to_token = AsyncOPChain.o_usdt, AsyncLiskChain.lsk
# tx = await AsyncSuperswap().swap(from_token, to_token, amount=1)
# print(tx)


# assert(tx.startswith("0x"))
# test_eq(relayer.get_call_count(), 1)
# last_call = relayer.get_last_call()
# test_eq(type(last_call["salt"]), str)
# test_eq(type(last_call["origin_domain"]), int)


#from_token, to_token = AsyncOPChain.o_usdt, AsyncLiskChain.lsk
#tx = Superswap().swap(from_token, to_token, amount=1)
    

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()